# packages

In [94]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("pandas")
install("numpy")
install("matplotlib")

# imports

In [95]:
import pandas as pd
import numpy as np
import json
import csv
import os

# Read Extraction Data from CSV

In [96]:
df = pd.read_csv(filepath_or_buffer="data/04 extraction consolidation results_3.csv",sep="|")

In [97]:
df

,Title,DT Definition,virtual space,Kritzinger,DT is specific,modeling language,language fidelity,language content category,language content category consolidated,note on modeling language,...,use case domain,system lifecycle phase,twinning target,twin lifecycle phase,purpose /expected benefit of using models,ModelPurposeCategory,open challenges,Technological Readiness Level,notes,Use Case
0,A framework for Model-Driven Engineering of re...,"dynamic virtual model of a system, processor, ...",just a model,DS,general,SysML,NaN,ontology,logical or physical,NaN,...,H Â— Transportation and Storage,Design,"Individual Systems, such as automated cars (At...",As-Designed,formal specification of information in the for...,automation,NaN,"1-3: die Technologie gibt's, evtl. am Toy Exam...",NaN,limited traffic zone access control
1,A framework for Model-Driven Engineering of re...,NaN,NaN,NaN,NaN,Knowledge base model,NaN,behavior,structure or behavior,contribution of the paper,...,NaN,Design,NaN,As-Designed,NaN,NaN,NaN,NaN,NaN,NaN
2,A Methodology for Digital Twin Modeling and De...,"""The digital twin (DT) is a virtual representa...",just a model,DM,general,AutomationML,NaN,ui configuraiton,"data, architecture, ontology, UI configuration",NaN,...,C Â— Manufacturing,Design,"Individual Systems, such as automated cars (At...",As-Designed,"model the physical device, serving as the virt...",abstraction,NaN,"1-3: die Technologie gibt's, evtl. am Toy Exam...",NaN,refinery automationsystem with four valves
3,A Methodology for Digital Twin Modeling and De...,NaN,NaN,NaN,NaN,NaN,NaN,data,NaN,NaN,...,NaN,Operate,"Systems of Systems, such as complete factories...",As-Operated,NaN,NaN,NaN,NaN,NaN,NaN
4,A Novel Methodology to Construct Digital Twin ...,"""a digital twin of a system consists of a set ...",Model + X,DT,general,Fault Tree,NaN,architecture,"data, architecture, ontology, UI configuration",NaN,...,"M Â— Professional, Scientific and Technical Ac...",Operate,"Individual Systems, such as automated cars (At...",As-Operated,NaN,NaN,NaN,"1-3: die Technologie gibt's, evtl. am Toy Exam...",NaN,ground station of a spacecraft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,TwinOps - DevOps Meets Model-Based Engineering...,NaN,NaN,NaN,NaN,NaN,NaN,ontology,"data, architecture, ontology, UI configuration",NaN,...,NaN,Implement,NaN,As-Designed,NaN,NaN,NaN,NaN,NaN,NaN
77,TwinOps - DevOps Meets Model-Based Engineering...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Implement,NaN,As-Designed,NaN,NaN,NaN,NaN,NaN,NaN
78,Understanding and Improving Model-Driven IoT S...,"""A digital twin (DT) of a system consists of a...",Model + X,DT,general,MontiThings,NaN,NaN,NaN,NaN,...,I Â— Accommodation and Food Service Activities,Operate,"Individual Systems, such as automated cars (At...",As-Operated,NaN,NaN,NaN,"1-3: die Technologie gibt's, evtl. am Toy Exam...",NaN,"smart lawn watering system; heating, ventilati..."
79,Understanding and Improving Model-Driven IoT S...,NaN,NaN,NaN,NaN,NaN,NaN,architecture,"data, architecture, ontology, UI configuration",NaN,...,NaN,Implement,NaN,As-Operated,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
result = df['Title'].to_list()

In [99]:
result = list(dict.fromkeys(result))
print(len(result))

36


In [100]:
titles = list(df.columns)

In [101]:
metadata = pd.read_csv("./data/search_results.csv", sep=";", encoding="latin-1")

In [102]:
metadata = metadata.drop_duplicates(subset=['Title'])

In [103]:
data = df.merge(metadata, how='left', on='Title')
#unique function

In [104]:
del metadata
del df
del titles

In [105]:
index = 0
print('{', end="")
for row in data:
    print(index, row)
    if index == 0:
        #print('\''+row+'\'', end="")
        index += 1
        continue
    #print(', '+'\''+row+'\'', end="")
    index += 1
print('}')

{0 Title
1 DT Definition
2 virtual space
3 Kritzinger
4 DT is specific
5 modeling language
6 language fidelity
7 language content category
8 language content category consolidated
9 note on modeling language
10 model type
11 model processing technique
12 model usage technique
13 source
14 source category
15 source sub-category
16 source model target category
17 target
18 target category
19 target sub-category
20 target model target category
21 purpose of MDE application
22 purpose of DT
23 DTPurposeCategory
24 use case domain
25 system lifecycle phase
26 twinning target
27 twin lifecycle phase
28 purpose /expected benefit of using models
29 ModelPurposeCategory
30 open challenges
31 Technological Readiness Level
32 notes
33 Use Case
34 Publication Title
35 Authors
36 Publication Year
37 Document Identifier
38 Publisher
}


# create JSON from CSV

In [106]:
model_trafos = 'modelTrafos'
modelling_language = 'modeling language'

In [107]:
keys = ['Title', 'DT_Definition', 'virtual space', 'Kritzinger', 'DT is specific', 'modeling language original', 'note on modeling language', 'model type original', 'modeling language', 'model type', 'model processing technique', 'model usage technique', 'source', 'target', 'purpose of MDE application', 'purpose of DT', 'DTPurposeCategory', 'use case domain', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'purpose /expected benefit of using models', 'open challenges', 'Technological Readiness Level', 'notes', 'Use Case', 'Publication Title', 'Authors', 'Publication Year', 'Document Identifier', 'Publisher']

In [108]:
out_menge_1 = {}
for key in result:
    out_menge_1[key] = {keys[1]:None,
                        model_trafos:{},
                        keys[22]:None,
                        keys[23]:None, 
                        keys[24]:None, 
                        keys[25]:None, 
                        keys[19]:None, 
                        keys[17]:None,
                        keys[2]:None,
                        keys[3]:None,
                        keys[4]:None,
                        modelling_language:{},
                        keys[15]:None,
                        keys[16]:None
                        }

In [109]:
data.reset_index()
for title in result:
    rownum = 0
    
    for index, row in data.iterrows():
        if title == row['Title'] and not pd.isna(title):
            rowcopy = row.__deepcopy__()
            
            if row['DT Definition'] is not None and not pd.isna(row['DT Definition']):
                out_menge_1[title]['DT_Definition'] = row['DT Definition']

            if row['purpose of DT'] is not None and not pd.isna(row['purpose of DT']):
                out_menge_1[title]['purpose of DT'] = row['purpose of DT']

            if row['DTPurposeCategory'] is not None and not pd.isna(row['DTPurposeCategory']):
                out_menge_1[title]['DTPurposeCategory'] = row['DTPurposeCategory']

            if row['use case domain'] is not None and not pd.isna(row['use case domain']):
                out_menge_1[title]['use case domain'] = row['use case domain']
            
            if row['twinning target'] is not None and not pd.isna(row['twinning target']):
                if out_menge_1[title]['twinning target'] is None:
                    out_menge_1[title]['twinning target'] = row['twinning target']
                else:
                    out_menge_1[title]['twinning target'] += ";"
                    out_menge_1[title]['twinning target'] +=  row['twinning target']

            if row['purpose /expected benefit of using models'] is not None and not pd.isna(row['purpose /expected benefit of using models']):
                out_menge_1[title]['purpose /expected benefit of using models'] = row['purpose /expected benefit of using models']

            if row['open challenges'] is not None and not pd.isna(row['open challenges']):
                out_menge_1[title]['open challenges'] = row['open challenges']

            if row['Technological Readiness Level'] is not None and not pd.isna(row['Technological Readiness Level']):
                out_menge_1[title]['Technological Readiness Level'] = row['Technological Readiness Level']
            
            if row['notes'] is not None and not pd.isna(row['notes']):
                out_menge_1[title]['notes'] = row['notes']

            if row['Use Case'] is not None and not pd.isna(row['Use Case']):
                out_menge_1[title]['Use Case'] = row['Use Case']

            if row['Publication Title'] is not None and not pd.isna(row['Publication Title']):
                out_menge_1[title]['Publication Title'] = row['Publication Title']
            
            if row['Authors'] is not None and not pd.isna(row['Authors']):
                out_menge_1[title]['Authors'] = row['Authors']
            
            if row['Publication Year'] is not None and not pd.isna(row['Publication Year']):
                out_menge_1[title]['Publication Year'] = row['Publication Year']

            if row['Document Identifier'] is not None and not pd.isna(row['Document Identifier']):
                out_menge_1[title]['Document Identifier'] = row['Document Identifier']

            if row['Publisher'] is not None and not pd.isna(row['Publisher']):
                out_menge_1[title]['Publisher'] = row['Publisher']
                
            if row['DT Definition'] is not None and pd.isna(row['DT Definition']):
                if row['virtual space'] is not None and not pd.isna(row['virtual space']):
                    out_menge_1[title]['virtual space'] = row['virtual space']
            
                if row['Kritzinger'] is not None and not pd.isna(row['Kritzinger']):
                    out_menge_1[title]['Kritzinger'] = row['Kritzinger']
            
                if row['DT is specific'] is not None and not pd.isna(row['DT is specific']):
                    out_menge_1[title]['DT is specific'] = row['DT is specific']
            
            if row['modeling language'] is not None and not pd.isna(row['modeling language']):
                out_menge_1[title]['modeling language'][row['modeling language']] = {'fidelity':row['language fidelity'],'category': row['language content category'],'consolidated_category':row['language content category consolidated'],'model_type':row['model type']}#.append(row['modeling language'])

            # multiple Trafos
            arrayindexes = [0,1,2,3,4,5,6,7,8,9,10,22,23,24,26,28,29,30,31,32,33,34,35,36,37,38]
            rowjson = rowcopy.drop(row.iloc[arrayindexes].index).to_json()
            out_menge_1[title][model_trafos][rownum] = rowjson
            #out_menge_1[title]['data'][rownum].append(rowjson)
            rownum+=1


In [110]:
del rowjson
del rowcopy
del result
del rownum

# export JSON to /json/data_consolidated_semistructured.json

In [111]:
json_object = json.dumps(out_menge_1, indent = 4) 
try:
    os.mkdir('target/')
    os.mkdir('target/img/')
except OSError as error:
    print(error)
with open("target/json/data_consolidated_semistructured__2.json", "w") as outfile:
    outfile.write(json_object)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'target/'
